# Import

In [27]:
import numpy as np
import pandas as pd

# Data preporation

In [28]:
# Load previous day data
file_name = 'release/covid19-russia-cases.csv'
rus_df = pd.read_csv(file_name)
rus_df.tail()

,Date,Region/City,Region/City-Eng,Region_ID,Day-Confirmed,Day-Deaths,Day-Recovered,Confirmed,Deaths,Recovered
5755,2020-06-10,Ханты-Мансийский АО,Khanty-Mansiysk,86.0,197.0,2.0,264.0,3821.0,23.0,1468.0
5756,2020-06-10,Чеченская Республика,Chechen Republic,95.0,16.0,0.0,49.0,1400.0,16.0,1073.0
5757,2020-06-10,Чукотский автономный округ,Chukotka Autonomous Okrug,87.0,2.0,0.0,1.0,112.0,1.0,69.0
5758,2020-06-10,Ямало-Ненецкий АО,Yamalo-Nenets Autonomous Okrug,89.0,54.0,0.0,7.0,2887.0,13.0,1482.0
5759,2020-06-10,Ярославская область,Yaroslavl region,76.0,63.0,0.0,142.0,3844.0,15.0,1452.0


In [29]:
# Read update data
# Result of data_parsing.ipynb
file_name = 'covid19-russia-cases-upd.csv'
upd_df = pd.read_csv(file_name)
upd_df.sort_values('Confirmed', ascending=False).head()

,Region/City,Confirmed,Date,Deaths,Recovered,Day-Confirmed,Day-Deaths,Day-Recovered
29,Москва,201221.0,2020-06-11,3138.0,115769.0,1436.0,53.0,2236.0
30,Московская область,47181.0,2020-06-11,644.0,14187.0,724.0,22.0,768.0
64,Санкт-Петербург,19769.0,2020-06-11,520.0,8864.0,303.0,38.0,343.0
33,Нижегородская область,13341.0,2020-06-11,152.0,6219.0,303.0,2.0,244.0
67,Свердловская область,8506.0,2020-06-11,48.0,3790.0,312.0,0.0,148.0


In [30]:
# Strip text data
rus_df['Region/City'] = rus_df['Region/City'].astype('str').str.strip('\u200b') 
upd_df['Region/City'] = upd_df['Region/City'].astype('str').str.strip('\u200b')

# Updating

In [31]:
def upd(row):
    reg = row['Region/City']
    
    row['Confirmed'] = rus_df[rus_df['Region/City'] == reg]['Confirmed'].max() + row['Day-Confirmed']
    row['Deaths'] = rus_df[rus_df['Region/City'] == reg]['Deaths'].max() + row['Day-Deaths']
    row['Recovered'] = rus_df[rus_df['Region/City'] == reg]['Recovered'].max() + row['Day-Recovered']
    
    row['Confirmed'] = row['Day-Confirmed'] if np.isnan(row['Confirmed']) else row['Confirmed']
    row['Deaths'] = row['Day-Deaths'] if np.isnan(row['Deaths']) else row['Deaths']
    row['Recovered'] = row['Day-Recovered'] if np.isnan(row['Recovered']) else row['Recovered']
    
    return row.drop('Region/City')

# If cumulative values unknown
# upd_df = upd_df.groupby('Region/City').apply(lambda df: upd(df.iloc[0])).reset_index()

In [32]:
# Date to datetime
upd_df['Date'] = pd.to_datetime(upd_df['Date'])
rus_df['Date'] = pd.to_datetime(rus_df['Date'])

In [33]:
# Region names translation
# from googletrans import Translator
# translator = Translator()

# def translate(rus):
#     eng = translator.translate(rus, src='ru', dest='en')
#     return eng.text

# upd_df['Region/City-Eng'] = upd_df['Region/City'].apply(translate)

In [34]:
# Add region ID and Eng columns
regions_df = pd.read_csv('release/regions-info.csv')

right = regions_df[['Region_ID', 'Region', 'Region_eng']]
upd_df = upd_df.merge(right, left_on='Region/City', right_on='Region', how='left')
upd_df['Region/City-Eng'] = upd_df.Region_eng
upd_df = upd_df[['Date', 'Region/City', 'Region/City-Eng', 'Region_ID', 'Day-Confirmed', 'Day-Deaths', 'Day-Recovered', 'Confirmed', 'Deaths', 'Recovered']]

In [35]:
rus_df = pd.concat([rus_df, upd_df]).reset_index(drop=True)
rus_df.tail(10)

,Date,Region/City,Region/City-Eng,Region_ID,Day-Confirmed,Day-Deaths,Day-Recovered,Confirmed,Deaths,Recovered
5835,2020-06-11,Тюменская область,Tyumen region,72.0,34.0,0.0,32.0,2047.0,12.0,1176.0
5836,2020-06-11,Удмуртская Республика,Udmurt republic,18.0,20.0,0.0,14.0,1031.0,20.0,516.0
5837,2020-06-11,Ульяновская область,Ulyanovsk region,73.0,88.0,0.0,106.0,3538.0,18.0,1201.0
5838,2020-06-11,Хабаровский край,Khabarovsk region,27.0,71.0,1.0,37.0,2804.0,19.0,1439.0
5839,2020-06-11,Ханты-Мансийский АО,Khanty-Mansiysk,86.0,186.0,1.0,91.0,4007.0,24.0,1559.0
5840,2020-06-11,Челябинская область,Chelyabinsk region,74.0,1.0,0.0,132.0,4130.0,37.0,2099.0
5841,2020-06-11,Чеченская Республика,Chechen Republic,95.0,20.0,0.0,46.0,1420.0,16.0,1119.0
5842,2020-06-11,Чукотский автономный округ,Chukotka Autonomous Okrug,87.0,2.0,0.0,1.0,114.0,1.0,70.0
5843,2020-06-11,Ямало-Ненецкий АО,Yamalo-Nenets Autonomous Okrug,89.0,77.0,0.0,3.0,2964.0,13.0,1485.0
5844,2020-06-11,Ярославская область,Yaroslavl region,76.0,59.0,0.0,78.0,3903.0,15.0,1530.0


In [36]:
# Negative values fixing
# Negative values can appear if some cases was removed from statistic
def negval_fix(reg):
    last = reg.iloc[-1]
    diff_c = last['Day-Confirmed']*(-1) if last['Day-Confirmed'] < 0 else 0
    diff_d = last['Day-Deaths']*(-1) if last['Day-Deaths'] < 0 else 0
    diff_r = last['Day-Recovered']*(-1) if last['Day-Recovered'] < 0 else 0
    
    last['Day-Confirmed'] += diff_c
    last['Day-Deaths']    += diff_d
    last['Day-Recovered'] += diff_r
    reg.iloc[-1] = last
    
    if diff_c + diff_d + diff_r > 0:
        print(f"Conf {diff_c} Death {diff_d} Rec {diff_r} {last['Region/City']}")
        prev = reg.iloc[-2]
        prev['Confirmed'] -= diff_c
        prev['Deaths']    -= diff_d
        prev['Recovered'] -= diff_r
        reg.iloc[-2] = prev
    
    return reg
    
rus_df = rus_df.groupby('Region/City').apply(negval_fix).reset_index(drop=True)

In [37]:
# Manual fixes if needed
# rus_df.loc[321, 'Confirmed'] = 5
# rus_df.loc[359, 'Day-Confirmed'] = 0


# rus_df.loc[3775, 'Deaths'] = 7
# rus_df.loc[3775, 'Day-Deaths'] = 0

# Data checking

In [38]:
rus_sum = rus_df.loc[rus_df['Region/City'] != 'Diamond Princess']

print('Russia Confirmed:', rus_sum.groupby('Region/City')['Confirmed'].max().sum(),
      rus_sum.groupby('Region/City')['Day-Confirmed'].sum().sum())
print('Russia Deaths:', rus_sum.groupby('Region/City')['Deaths'].max().sum(),
      rus_sum.groupby('Region/City')['Day-Deaths'].sum().sum())
print('Russia Recovered:', rus_sum.groupby('Region/City')['Recovered'].max().sum(),
      rus_sum.groupby('Region/City')['Day-Recovered'].sum().sum())

rus_regs = rus_sum.groupby('Region/City')['Confirmed'].max().reset_index()
rus_regs['Confirmed-ByDay'] = rus_sum.groupby('Region/City')['Day-Confirmed'].sum().reset_index(drop=True)
print(rus_regs.loc[rus_regs['Confirmed'] != rus_regs['Confirmed-ByDay']])

rus_regs = rus_sum.groupby('Region/City')['Recovered'].max().reset_index()
rus_regs['Recovered-ByDay'] = rus_sum.groupby('Region/City')['Day-Recovered'].sum().reset_index(drop=True)
print(rus_regs.loc[rus_regs['Recovered'] != rus_regs['Recovered-ByDay']])

rus_regs = rus_sum.groupby('Region/City')['Deaths'].max().reset_index()
rus_regs['Deaths-ByDay'] = rus_sum.groupby('Region/City')['Day-Deaths'].sum().reset_index(drop=True)
print(rus_regs.loc[rus_regs['Deaths'] != rus_regs['Deaths-ByDay']])

# rus_regs.to_csv('rus_regs.csv')

Russia Confirmed: 502436.0 502436.0
Russia Deaths: 6559.0 6559.0
Russia Recovered: 261173.0 261173.0
Empty DataFrame
Columns: [Region/City, Confirmed, Confirmed-ByDay]
Index: []
Empty DataFrame
Columns: [Region/City, Recovered, Recovered-ByDay]
Index: []
Empty DataFrame
Columns: [Region/City, Deaths, Deaths-ByDay]
Index: []


# Data saving

In [39]:
file_name = 'release/covid19-russia-cases.csv'
rus_df.to_csv(file_name, index=False)